# Computação Gráfica 2025/2026

Um tutorial de Computação Gráfica por André Falcão  - (C) Docentes MC-DSI, FCUL- 2020-2025

## TP02 - Representações e transformações Geométricas em 2D

Primeiro vamos importar as bibliotecas conhecidas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Agora vamos criar uma função genérica de desenho de polígonos em matplotlib. Vamos fazer o nosso *viewport* entre `[-5, 5]` para os 2 eixos. PAra jjá o Viewport está nas coordenadas do mundo (*world coordinates*)

In [ ]:
# desenha vários polígonos polígono
def desenha_poligonos(P_list, title="", figsize=(5,5), xlim=(-5,5), ylim=(-5,5)):
    plt.figure(figsize=figsize)
    all_polys=[]
    for P, label in P_list:
        Q = np.vstack([P, P[0]]) #acrescenta o primeiro vértice ao fim da lista
        plt.plot(Q[:,0], Q[:,1], marker='o', label=label)
        all_polys.append(P)
        
    all_pts = np.vstack(all_polys)
    min_xy = all_pts.min(axis=0)[:2] #this is nice for homogeneous coordinates later
    max_xy = all_pts.max(axis=0)[:2] 
    pad = 0.15 * np.linalg.norm(max_xy - min_xy) if np.any(max_xy - min_xy) else 1.0
    plt.axhline(0, linewidth=1)
    plt.axvline(0, linewidth=1)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.grid(True, linestyle=':')
    #plt.xlim(min_xy[0]-pad, max_xy[0]+pad)
    #plt.ylim(min_xy[1]-pad, max_xy[1]+pad)
    plt.xlim(xlim[0], xlim[1])
    plt.ylim(*ylim)
    plt.title(title)
    plt.legend()
    plt.show()

Vamos definir agora um polígono irregular simples, que caiba no viewport definido

**NOTA:** Repare-se que nesta aula vamos considerar os pontos sob a forma de 2 colunas (1 para o X e a 2a coluna para o y) Assim as nossas operações serão feitas da Direita para a esquerda, com os Pontos a aparecerem primeiro e depois as operações matriciais sequencialmente.  Isto é diferente da ordem apresentada nas aulas teóricas em que os pontos foram apresentados com 2 linhas, uma para cada eixo.

In [ ]:


P = np.array([
    [-1.0, -1.0],
    [ 1.3, -0.7],
    [ 2.0,  1.0],
    [ 3.0, -0.2],
    [ 1.3, -1.5]
])

P = np.array([
    [2.0, 0.0],
    [3.0, 0.0],
    [3.0, 1.0],
    [2.5, 2.0],
    [2.0, 1.0]
])



desenha_poligonos(zip([P], ["Original"]),"Polígono original")


### Translacções

O nosso propósito é definir transformações simples em 2D. Vamos começar com a translação que o que temos que fazer é apenas somar um vector aos pontos

In [ ]:

def translate(P, tx, ty):
    """Desloca pontos Nx2 por (tx, ty)"""
    return P + np.array([tx, ty])


In [ ]:
P_t =translate(P, -3.5,-3)
desenha_poligonos(zip([P, P_t], ["Original", "Translacao"]),"Translacção")

### Rotações

In [ ]:
#cria matriz de Rotação
def A_rotate(theta_deg):
    th = np.deg2rad(theta_deg)
    c, s = np.cos(th), np.sin(th)
    return np.array([[c, -s],
                     [s,  c]])

def rotate_around_origin(P, theta_deg):
    """Roda pontos Nx2 em torno da origem (0,0)."""
    A = A_rotate(theta_deg)   # matriz 2x2 de rotação
    return P @ A.T #<-Multiplica pontos pela matriz de rotação



In [ ]:
P_r =rotate_around_origin(P, 70)
desenha_poligonos(zip([P, P_r], ["Original", "Rotacao"]),"Rotação")

### Escalamento

In [ ]:

#matriz de escalamento
def A_scale(sx=1.0, sy=1.0):
    return np.array([[sx, 0.0],
                     [0.0, sy]])


#PODE SER UM EXERCÍCIO
def scale(P, sx, sy):
    A = A_scale(sx, sy)   # matriz 2x2 de escalamento
    return P @ A.T        #<-Multiplica pontos pela matriz de escalamento


In [ ]:
sx, sy = 1.5, 1.5
P_s = scale(P, sx, sy)
desenha_poligonos(zip([P, P_s], ["Original", "Escalado"]),"Escalamento")


### Rotações em torno de um eixo arbitrário

vamos fazer agora uma rotação pelo centro de massa do nosso objecto. PAra já vamos precisar de uma função que o calcule

In [ ]:
def centroid(P):
    return P.mean(axis=0)

Agora como vimos na teórica, o procedimento é:
1. Translaccionar os pontos para um novo referencial (subtrair o centroide
2. Fazer a Rotação
3. Repor o objecto na posição original

In [ ]:
theta = 75  # graus

# calcular centroide
C = centroid(P)
P_t= translate(P, -C[0], -C[1])
P_tr = rotate_around_origin(P_t, theta) 
P_trt= translate(P_tr, C[0], C[1])

desenha_poligonos(zip([P, P_trt], ["Original", "Rot Centroid"]),"Rotação no centroide")



#### Exercícios 1

1. Crie uma função `rotate` que faça uma rotação genérica à volta de um eixo qualquer e que receba como argumentos, o polígono, o ângulo e o eixo de rotação
2. Crie uma função `rotate_centroid` que faça a rotação à volta do centroid para um polígono genérico
3. Comente o desempenho destas funções para malhas poligonais muito grandes
4. Uma outra transformação geométrica designada cizalhamento, (shear), faz uma alteração de escala dos eixos dependendo dos outros eixos e pode ser caracterizada como


$$
\begin{bmatrix}
x' \\
y'
\end{bmatrix}
=
\begin{bmatrix}
1 & k_x \\
k_y & 1
\end{bmatrix}
\begin{bmatrix}
x \\
y
\end{bmatrix}
$$

    Crie uma função `shear` que receba como argumentos as escalas do sizalhamento e represente o que aconteceria ao nosso polígono depois antes e depois


## Coordenadas Homogéneas

Uma forma de melhorarmos o desempenho e não estarmos a aplicar repetidamente as mesmas transformações sucessivamente é fazermos composições de matrizes (multiplicações) sobre as transformações que iremos aplicar

Para isso necessitaremos de adicionar uma coluna aos nossos pontos e transformar todas as matrizes em 


**Translação (2D, homogéneo):**
$$
\begin{bmatrix}x'\\y'\\1\end{bmatrix}
=
\begin{bmatrix}
1 & 0 & t_x\\
0 & 1 & t_y\\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}x\\y\\1\end{bmatrix}
$$

**Rotação por $\theta$ (2D, homogéneo):**
$$
\begin{bmatrix}x'\\y'\\1\end{bmatrix}
=
\begin{bmatrix}
\cos\theta & -\sin\theta & 0\\
\sin\theta & \cos\theta & 0\\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}x\\y\\1\end{bmatrix}
$$

**Escalamento (2D, homogéneo):**
$$
\begin{bmatrix}x'\\y'\\1\end{bmatrix}
=
\begin{bmatrix}
s_x & 0 & 0\\
0 & s_y & 0\\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}x\\y\\1\end{bmatrix}
$$

Nesta aula, contudo uma vez que os pontos estão represntados em linhas será necessário transpor a matriz de pontos antes da transformação, e após a transformação, colocar os novos pontos no mesmo formato para  a função de desenho

Para já precisaremos também de uma função que coloque qualquer matriz de pontos no formato homogéneo (com mais uma coluna de 1s)


In [ ]:
#esta função aplica uma Matriz de transformação a uma matriz com vértices 2D em coordenadas homogéneas
def apply_transform(P, M): 
    return (M @ P.T).T


#aqui convertiremos os nossos pontos 2D para coordenadas homogéneas
def to_homog(P):
    """Nx2 -> Nx3 (homogéneo)"""
    return np.c_[P, np.ones(len(P))]



to_homog(P)

Podemos agora definir funções que criem as várias matrizes, como definidas acima, para depois as podermos compor sob a forma de multiplicações de matrizes

In [ ]:
def M_translate(tx, ty):
    return np.array([[1, 0, tx],
                     [0, 1, ty],
                     [0, 0,  1]], dtype=float)


def M_rotate_origin(theta_deg):
    th = np.deg2rad(theta_deg)
    c, s = np.cos(th), np.sin(th)
    R = np.array([[ c, -s, 0],
                  [ s,  c, 0],
                  [ 0,  0, 1]], dtype=float)
    return R


def M_scale(sx, sy):
    return np.array([[sx, 0, 0],
                     [0, sy, 0],
                     [0, 0,  1]], dtype=float)





### Translação em coordenadas homogéneas

In [ ]:
P_h=to_homog(P)

M = M_translate(-3.5, -3)
P_t =apply_transform(P_h, M)
desenha_poligonos(zip([P_h, P_t], ["Original", "Translacao"]),"Translacção")

### Rotação em coordenadas homogéneas

Vamos fazer uma rotação sobre a origem

In [ ]:

M = M_rotate_origin(75)
P_t =apply_transform(P_h, M)
desenha_poligonos(zip([P_h, P_t], ["Original", "Rotação"]),"Rotação na origem")

### Escalamento

Faremos agora um escalamento. Note-se o que acontece: apesar do obejcto ser apenas escalado, aparenta deslocar-se

In [ ]:

M = M_scale(1.6,1.6)
P_t = apply_transform(P_h, M)
desenha_poligonos(zip([P_h, P_t], ["Original", "Escalamento"]),"Escalamento %3.1f" % (1.6))

### Rotação sobre si próprio

Agora faremos uma rotação sobre si próprio aplicando 3 multiplicações de matrizes:
1. Translação do centroide para a origem
2. Rotação
3. Translação da origem para o centroide original

In [ ]:
# calcular centroide
C = centroid(P_h)

#Calcular as matrizes para a composição geométrica
Mt1 = M_translate(-C[0], -C[1])
Mr  = M_rotate_origin(75)
Mt2 = M_translate( C[0],  C[1])

#ESTA É A PARTE IMPORTANTE!
#Atenção à ordem a que estamos a fazer a multiplicação das matrizes
M   = Mt2 @ Mr @ Mt1 

#agora sim, transformamos os pontos da nossa cena
P_t = apply_transform(P_h, M)
desenha_poligonos(zip([P_h, P_t], ["Original", "Rotação"]),"Rotação no centroide")

#### Exercícios 2

1. Crie uma função `M_rotate`capaz de produzir uma matriz de rotação genérica e que receba 2 argumentos: o ângulo de rotação e o ponto de rotação. Verifique os resultados
2. Crie uma função `M_rotate_centroid` que use a anterior e faça uma rotação sobre o centroide, recebendo apenas um argumento: o ângulo de rotação). Verifique os resultados
3. Aplique as seguintes transformações compostas, criando as matrizes e verificando os resultados
    1. escalamento (0.5), rotação na origem de 120 graus, e translação de (-2, -2)
    2. Deslocação do centroide para a origem, escalamento (2x) e rotação de 85 graus
    3. rotação no eixo de (-2, 2) de -43 graus, translação de (1,2)  e rotação sbre o centroide de 165 graus

    Para todas as composições verifique os resultados e se estão consistentes com o que esperaria.
4. Represente no mesmo gráfico os resultados das 3 transformações anteriores e do polígono original, com legendas adequadas
5. [OPCIONAL] Para todos os exercícios do ponto anterior, experimente representar no mesmo gráfico o polígono em todas as fase do processos
6. Comente os resultados, relativamente a a) desempenho face às operações anteriores; b) facilidade de implementação; c) escalabilidade 


# Transformações do Mundo para o Viewport

Queremos mapear uma cena em coordenadas do Mundo (**world window**) numa vista do ecrã ( **viewport**).

Assumindo que o nosso mundo está entre 0 e 3, mas queremos uma janela nas coordenadas do mundo de
  $$
  x \in [x_{\min}, x_{\max}] = [1.8, 2.5], \quad 
  y \in [y_{\min}, y_{\max}] = [0, 0.5]
  $$

E queremos projectar para o **Viewport:**  
  $$
  x_v \in [0, 640], \quad y_v \in [0, 480]
  $$


Para isso temos 3 etapas:
1. Determinar as dimensões das janelas respectivas e calcular as escalas
2. Definir as Transformações individuais (Translações e Rotação)
3. Calcumar a Matriz que transforma todos os poligonos da cena nas coordenadas do Viewport

#### 1. Determinar as dimensões

$$
W_x = 2.5 - 1.8 = 0.7, \quad W_y = 0.5 - 0 = 0.5
$$  

$$
V_x = 640, \quad V_y = 480
$$

Os **factores de escala** serão então:

$$
S_x = \frac{V_x}{W_x} = \frac{640}{0.7} \approx 914.29, \quad 
S_y = \frac{V_y}{W_y} = \frac{480}{0.5} = 960
$$



#### 2.  Definir as matrizes das transformações


1. **Translação de coordenadas do mundo para a origem:**  
$$
T_1 = \begin{bmatrix}
1 & 0 & -1.8 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

2. **Escalar para a dimensão do viewport**
$$
S = \begin{bmatrix}
S_x & 0 & 0 \\
0 & S_y & 0 \\
0 & 0 & 1
\end{bmatrix}
=
\begin{bmatrix}
914.29 & 0 & 0 \\
0 & 960 & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

3. **Translação para o mínimo do viewport (aqui é \((0,0)\)):**  
$$
T_2 = I = \begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
$$



#### 3.Calcular a transformação global (assumindo que o yy 'sobe')

$$
M = T_2 \, S \, T_1 =
\begin{bmatrix}
914.29 & 0 & -1645.71 \\
0 & 960 & 0 \\
0 & 0 & 1
\end{bmatrix}
$$




Vamos para já criar uma cena com 2 objectos, usando o viewport definido por omissão. Podemos ver que estes objectos são muito pequenos (variação muito pequena em x e y) tendo que ser ajustados para um novo viewport

In [ ]:
PW1 = np.array([
     [2.0, 0.0],
     [2.1, 0.0],
     [2.1,  .1],
     [2.05, .2],
     [2.0,  .1]
], dtype=float)

PW2 = np.array([
     [2.20, 0.0],
     [2.21, 0.0],
     [2.21, .15],
     [2.26, .20],
     [2.20, .25],
     [2.15, .20],
     [2.2, .15]
], dtype=float)


#colocar os objectos em coordenadas homogéneas
PW1_h=to_homog(PW1)
PW2_h=to_homog(PW2)

desenha_poligonos(zip([PW1_h, PW2_h], ["Poligono1", "Poligono2"]), "poligonos no espaço do mundo")

Primeiro vamos definir calcular a informação relevante para a projecção nas coordenadas do mundo num novo viewport de (640x480)

In [ ]:
vp_min_x, vp_max_x = 0, 640
vp_min_y, vp_max_y = 0, 480

#canto inferior esquerod do mundo
WC_LL = np.array([1.80, 0])

#Canto superior direito
WC_UR = np.array([2.5,0.5])

#dimensão da janela do mundo (x e y)
W_x, W_y = WC_UR - WC_LL

Agora vamos calcular as matrizes  e  aplicar as transformações e verificar se o resultado da matriz calculada é o esperado

In [ ]:
#1a trransformação <- translação para o 0,0 novo
Mt1 = M_translate(-WC_LL[0], -WC_LL[1])

#2a transformação, calcular as escalas respectivas
sx = (vp_max_x- vp_min_x)/W_x
sy = (vp_max_y- vp_min_y)/W_y
Ms  = M_scale(sx, sy)

#3a transformação - colocar no (0,0) do viewport)
Mt2 = M_translate(vp_min_x, vp_min_y)

#calcular transformação Final
M   =  Mt2 @ Ms @ Mt1 

#e podemos visualizar a transformação final
np.set_printoptions(precision=3,suppress=True)
M


Finalmente podemos projectar os dois objectos no novo referencial

In [ ]:

#Apicar a transformação final aos 2 polígonos
P1_t = apply_transform(PW1_h, M)
P2_t = apply_transform(PW2_h, M)

#desenha_poligonos(zip([P_t], ["Original"]),"Rotação no centroide")
desenha_poligonos(zip([P1_t, P2_t], ["Poligono1", "Poligono2"]),"Projecção no Viewport", 
                  xlim=(vp_min_x, vp_max_x), 
                  ylim=(vp_min_y, vp_max_y))

#### Exercícios

1. Mude o referencial do viewport para (100, 640) e (100, 480). Antes de executar, o que acha que irá acontecer?
2. E se o Canto inferiod das coordenadas do mundo passar para (1.9, 0)? e para (1.5,0)?
3. Rode a cena nas coordenadas do mundo 5 graus sobre a origem e represente-o no viewport de 640x480
    1. Em primeiro lugar calcule a matriz de transformação final
    2. Verifique se o resultado é o que pretendia
    3. Comente os seus resultados?
4. Rode a cena original (2 objectos) sobre o seu centroide - comum a ambos os polígonos - num ângulo de -7 graus e represente o resultado no mesmo viewport anterior
    1. calcule a matriz de transformação final
    2. Verifique se o resultado é o que pretendia